In [1]:
# Install required libraries
!pip install fastapi uvicorn nest-asyncio pyngrok
!pip install torch bitsandbytes
!pip install git+https://github.com/huggingface/accelerate
!pip install qwen-vl-utils[decord]==0.0.8
!pip install langchain_groq jsbeautifier

  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-x39s7u9r
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-x39s7u9r
  Resolved https://github.com/huggingface/accelerate to commit 9015a26f09027a2b82898f07a6ea251df6ed5c68
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
# Import necessary libraries
import nest_asyncio
import uvicorn
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage
import jsbeautifier
import json
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor, BitsAndBytesConfig
from qwen_vl_utils import process_vision_info
from PIL import Image
import torch
import re
import os
from io import BytesIO


# Set Groq API Key
os.environ["GROQ_API_KEY"] = "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"  # Replace with your Groq API Key


# ---- Load Qwen Model & Processor ----

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype='bfloat16'
)

model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-7B-Instruct",
    device_map="auto",
    quantization_config=bnb_config
)

processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct")



config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/57.6k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [3]:
!pip install python-multipart flask_restful

In [4]:
!pip install flask-cors

In [7]:
import nest_asyncio
from flask import Flask, request, jsonify
from flask_restful import Api, Resource
from PIL import Image
from io import BytesIO
import threading
import re
from flask_cors import CORS
import json
import jsbeautifier
import base64
from langchain_groq import ChatGroq
from langchain.schema.messages import HumanMessage, SystemMessage

# ---- Helper Functions ----
fn_solve_equation = [
    {
        "name": "SolveEquation",
        "description": "Solve the following mathematical equation(s) in a detailed, step-by-step manner. The equations may involve advanced mathematical concepts, including algebra, calculus, trigonometry, logarithms, integrals, or differential equations. If there are multiple equations, they will be separated by commas. Show all necessary steps, simplify where possible, and provide the final solution clearly for equation(s).",
        "parameters": {
            "type": "object",
            "properties": {
                "status": {
                    "type": "string",
                    "description": "Solution status. 'solved' if solved, 'cannot be solved' otherwise."
                },
                "solution": {
                    "type": "string",
                    "description": "The solution of the equation, if solvable."
                },
                "explanation": {
                    "type": "string",
                    "description": "Brief explanation of the solution."
                }
            },
            "required": ["status", "solution", "explanation"]
        }
    }
]

def output2json(output):
    opts = jsbeautifier.default_options()
    return json.loads(jsbeautifier.beautify(output["tool_calls"][0]["function"]["arguments"], opts))

import re

def extract_clean_equation(output_text):
    try:
        raw_output = output_text[0].strip()
        raw_output = re.sub(r"```.*?```", "", raw_output, flags=re.DOTALL).strip()
        raw_output = re.sub(r"(\$\$|\\\[|\\\]|\\\(|\\\))", "", raw_output).strip()
        match = re.search(r"extracted_equation\s*[:=]\s*[\"'](.*?)[\"']", raw_output)
        if match:
            equation = match.group(1)
        else:
            equation = raw_output
        return equation
    except Exception as e:
        return None

def solve_equation(equation):
    system_prompt = """
    You are a highly knowledgeable mathematical problem solver.
    Given one or more mathematical equations, perform the following:
    1. Carefully analyze each equation.
    2. If the equation is already solved (i.e., the solution is explicitly given), leave it and do not attempt to solve.
    3. Otherwise, solve the equation in a clear, step-by-step manner. Include simplification steps, use appropriate algebraic, trigonometric, calculus, logarithmic, or differential techniques as required.
    4. Present the final simplified solution clearly.
    5. Return the solution, a 'status' field as 'solved' or 'already solved', and a brief explanation of the approach taken.

    Provide your output strictly in the following function call format.
    """
    user_prompt = f"Equation: {equation}"
    llm = ChatGroq(
        model="deepseek-r1-distill-llama-70b",
        temperature=0.3,
    )
    completion = llm.predict_messages(
        [
            SystemMessage(content=system_prompt),
            HumanMessage(content=user_prompt),
        ],
        functions=fn_solve_equation,
    )
    output_analysis = completion.additional_kwargs
    json_output = output2json(output=output_analysis)
    return json_output

# ---- Flask Setup ----
nest_asyncio.apply()
app = Flask(__name__)
CORS(app)  # <-- Enable CORS for all origins
api = Api(app)

# ---- Flask Resource ----
class ExtractSolve(Resource):
    def post(self):
        try:
            # Get uploaded image
            file = request.files['file']
            contents = file.read()

            # Convert to base64
            encoded_image = base64.b64encode(contents).decode('utf-8')

            # Prepare message for model
            messages = [
                  {
                      "role": "user",
                      "content": [
                          {
                              "type": "image_url",
                              "image_url": f"data:image/jpeg;base64,{encoded_image}"
                          },
                          {
                              "type": "text",
                              "text": "Extract all the mathematical equations from the provided image. The equations may be written on rough paper, with imperfections or uneven handwriting, but they contain only mathematical expressions, variables, numbers, and standard math symbols. Recognize all digits accurately (ensure that zeros 0 are recognized as zero, not circles or other symbols). Return each equation on a single line. If there are multiple equations, separate them with commas. Do NOT include any extra text, only the equations. Also Recheck it twice before giving output and count also no. of equations and terms are equal"
                          }
                      ]
                  }
              ]


            # Process inputs
            text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            image_inputs, video_inputs = process_vision_info(messages)
            inputs = processor(
                text=[text],
                images=image_inputs,
                videos=video_inputs,
                padding=True,
                return_tensors="pt"
            )
            inputs = inputs.to("cuda")

            # Inference
            generated_ids = model.generate(**inputs, max_new_tokens=128)
            generated_ids_trimmed = [
                out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
            ]
            output_text = processor.batch_decode(
                generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
            )

            # Extract equation
            equation = extract_clean_equation(output_text)
            if not equation:
                return jsonify({"status": "error", "message": "Equation extraction failed."})

            # Solve equation
            groq_response = solve_equation(equation)

            return jsonify({
                "status": "success",
                "extracted_equation": equation,
                "solution_status": groq_response.get("status"),
                "solution": groq_response.get("solution"),
                "explanation": groq_response.get("explanation")
            })

        except Exception as e:
            return jsonify({"status": "error", "message": str(e)})

# Add Resource
api.add_resource(ExtractSolve, '/extract_solve/')

# ---- Run Server ----
def run():
    app.run(host='0.0.0.0', port=8003)

thread = threading.Thread(target=run, daemon=True)
thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


In [8]:
# Install & Setup Cloudflare Tunnel
!curl -L https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -o cloudflared
!chmod +x cloudflared

# Start Cloudflare Tunnel
!./cloudflared tunnel --url http://localhost:8003

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 36.0M  100 36.0M    0     0  27.1M      0  0:00:01  0:00:01 --:--:-- 89.6M
2025-03-22T12:19:34Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
202

INFO:werkzeug:127.0.0.1 - - [22/Mar/2025 12:19:54] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [22/Mar/2025 12:26:09] "POST /extract_solve/ HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Mar/2025 12:26:25] "POST /extract_solve/ HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Mar/2025 12:26:37] "POST /extract_solve/ HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Mar/2025 12:26:55] "POST /extract_solve/ HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Mar/2025 12:27:13] "POST /extract_solve/ HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Mar/2025 12:27:31] "POST /extract_solve/ HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Mar/2025 12:28:06] "POST /extract_solve/ HTTP/1.1" 200 -


2025-03-22T12:28:34Z INF Initiating graceful shutdown due to signal interrupt ...
^C
